# Mock mid-term exam

## Topic of the exam: Pre-BREXIT analysis of Non-metallic Minerals vs. Metal Ores. 

Please ensure that you run each cell (ctrl+enter) so that your inputs are saved 

All questions have either coding cell or a text cell, or a combination of the two. 

You are allowed to create additional cells for each answer to facilitate your work  

## Data preparation: Download and import exiobase

Data to be used in the exam: **year 2016 industry-by-industry format**

Link to the data: https://zenodo.org/record/5589597

In [1]:
# Import modules
import pandas as pd
import numpy as np

In [2]:
# Import here your data
path = "data/IOT_2016_ixi/"
A = pd.read_csv(f'{path}A.txt', sep='\t', index_col=[0, 1], header=[0, 1])
Y = pd.read_csv(f'{path}Y.txt' , sep='\t', index_col=[0, 1], header=[0, 1])

In [3]:
# Import satellite accounts
F_sat = pd.read_csv(f'{path}satellite/F.txt' , sep='\t', index_col=[0], header=[0, 1])
F_sat_hh = pd.read_csv(f'{path}satellite/F_y.txt' , sep='\t', index_col=[0], header=[0, 1])

## Question 1: Territorial accounting (10 points)

### Q1.1 What was the UK's domestic extractions of non-metallic minerals in 2016? (10 points) 
Hint 1: use the environmental extensions with keywords: "Domestic Extraction Used - Non-Metallic Minerals"

Hint 2: UK's acronym is GB (i.e., Great Britain) in ISO3 

In [4]:
F_minerals_ = F_sat.loc[F_sat.index.str.contains("Domestic Extraction Used - Non-Metallic Minerals")]
F_minerals_hh = F_sat_hh.loc[F_sat_hh.index.str.contains("Domestic Extraction Used - Non-Metallic Minerals")]
F_minerals_tot_ter = pd.concat([F_minerals_.loc[:,"GB"], F_minerals_hh.loc[:,"GB"]], axis=1).sum().sum()
F_minerals_tot_ter

204114.6671573314

## Question 2: Footprint analysis (10 points)

### Q2.1 What was the UK's non-metallic mineral footprint in 2016? (10 points)

In [5]:
# Calculate the rest of MRIO variables
I = np.identity(A.shape[0])
L = np.linalg.inv(I-A)
x = L @ Y.sum(axis=1)

In [6]:
x_ = x.copy()
x_[x_!=0] = 1/x_[x_!=0]
inv_diag_x_ = np.diag(x_)

In [7]:
# Intensities of non-metallic minerals 
f_minerals = F_minerals_.sum() @ inv_diag_x_ 

In [8]:
Y_reg = Y.groupby(level=0, axis=1, sort=False).sum()

In [9]:
e_minerals = f_minerals @ L @ Y_reg.loc[:, "GB"] + F_minerals_hh.loc[:, "GB"].sum().sum()
e_minerals

291457.89122951834

## Questions 3: Trade (15 points)

### Q3.1: Based on Q1 & Q2, was the UK a net importer or net exporter of non-metallic minerals in 2016? (3 points)

In [10]:
# Insert here your calculations
net_import = e_minerals - F_minerals_tot_ter
net_import

87343.22407218695

The UK was a net importer of non-metallic minerals in 2016

### Q3.2: Perform MRIO calculation and identify the top three regions from which the UK imported non-metallic minerals in 2016 (12 points)

In [11]:
e_minerals = np.diag(f_minerals) @ L @ Y_reg

e_minerals.index = Y.index

region_labels = Y.index.to_frame(index=False).region.unique()
sector_labels = Y.index.to_frame(index=False).sector.unique()

CBA_minerals = pd.DataFrame(e_minerals.loc[:, "GB"].values.reshape(49,163), columns=sector_labels, index=region_labels).sum(axis=1)

CBA_minerals[CBA_minerals.index!="GB"].sort_values(ascending=False).head(3)

IN    33566.719712
CN    27925.048258
WA    20235.464480
dtype: float64

## Question 4: Dashboard analysis - Non-metallic Minerals vs. Metal Ores (25 points)

### Q4.1 Quantify the reliance of the UK on each of the two global regions, EU27 and Non-EU27, in 2016 concerning Non-Metallic minerals and Metal Ores. (10 points)

Hint 1: Replicate Qs 2-3 for metal ores

Hint 2: use the environmental extensions with keywords: "Domestic Extraction Used - Metal Ores"

Hint 3: remember that the first 27 regions in exiobase are the EU27

Hint 4: use the last 21 countries/regions in exiobase for Non-EU27

In [12]:
F_metals_ = F_sat.loc[F_sat.index.str.contains("Domestic Extraction Used - Metal Ores")]
F_metals_hh = F_sat_hh.loc[F_sat_hh.index.str.contains("Domestic Extraction Used - Metal Ores")]

# Total territorial emissions
F_metals_tot_ter = F_metals_.loc[:,"GB"].sum().sum() + F_metals_hh.loc[:,"GB"].sum().sum()
F_metals_tot_ter

18.20531396715915

In [13]:
geo_e_minerals = e_minerals.groupby(level=0, axis=0, sort=False).sum()

In [14]:
geo_e_minerals.GB.iloc[:27].sum()

58738.837395005765

In [15]:
minerals_UK_CBA = geo_e_minerals.loc[:, "GB"] 
minerals_UK_PBA = geo_e_minerals.loc["GB", :]

minerals_UK_trade_rel = minerals_UK_CBA - minerals_UK_PBA

minerals_UK_trade_rel_EU = minerals_UK_trade_rel.iloc[:27].sum()
minerals_UK_trade_rel_ROW = minerals_UK_trade_rel.iloc[-21:].sum()

UK_dependency_minerals = pd.DataFrame([minerals_UK_trade_rel_EU, minerals_UK_trade_rel_ROW], index=["UK_dependency_on_EU","UK_dependency_on_ROW"], columns=["non_metallic_minerals, kt"]) 

UK_dependency_minerals


,"non_metallic_minerals, kt"
UK_dependency_on_EU,14631.080459
UK_dependency_on_ROW,72712.143613


In [16]:
# Intensities of metals ores 
f_metals = F_metals_.sum() @ inv_diag_x_ 

In [17]:
e_metals = np.diag(f_metals) @ L @ Y.groupby(level=0, axis=1, sort=False).sum()
e_metals.index = Y.index

In [18]:
geo_e_metals = e_metals.groupby(level=0, axis=0, sort=False).sum()

In [19]:
metals_UK_CBA = geo_e_metals.loc[:, "GB"] 
metals_UK_PBA = geo_e_metals.loc["GB", :]

metals_UK_trade_rel = metals_UK_CBA - metals_UK_PBA

metals_UK_trade_rel_EU = metals_UK_trade_rel.iloc[:27].sum()
metals_UK_trade_rel_ROW = metals_UK_trade_rel.iloc[-21:].sum()


UK_dependency_metals = pd.DataFrame([metals_UK_trade_rel_EU, metals_UK_trade_rel_ROW], index=["UK_dependency_on_EU","UK_dependency_on_ROW"], columns=["metal_ores, kt"]) 

UK_dependency_metals


,"metal_ores, kt"
UK_dependency_on_EU,11985.258124
UK_dependency_on_ROW,68503.823830


In [20]:
# Depedency dashboard net imports from EU27 and ROW
dashboard = pd.concat([UK_dependency_minerals, UK_dependency_metals],axis=1)
dashboard

,"non_metallic_minerals, kt","metal_ores, kt"
UK_dependency_on_EU,14631.080459,11985.258124
UK_dependency_on_ROW,72712.143613,68503.823830


In [21]:
# Dependency dashboard net imports from EU27 and ROW relative to total footprint 

dashboard_relative = round(dashboard / np.array([minerals_UK_CBA.sum(), metals_UK_CBA.sum()]) *100, 2)
dashboard_relative.columns = ["non_metallic_minerals, %", "metal_ores, %"]
dashboard_relative

,"non_metallic_minerals, %","metal_ores, %"
UK_dependency_on_EU,5.02,14.89
UK_dependency_on_ROW,24.95,85.09


### Q4.2 Focusing on the two types of materials, describe three significantly different findings about the UK's material reliance on the EU27 vs the rest of the world. (15 points)

The description for each finding should:

1) be supported by numerical results you obtained from above calculations and based on comparisons of numerical results; 

2) contains your reflections about the UK's trade relationship in the context of BREXIT.


_write your answer here_